In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104545/John-Isner&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2023 US Open R64
L vsMmoh
48.1%
63.0%
55.7%
55.6%
62.2%
77.5%
64.3%
73.1%
57.1%
40.3%
0.0%
19.6%
34.8%
45.9%
69
2023 US Open R128
W vsDiaz Acosta
44.6%
65.1%
65.2%
56.3%
43.4%
64.2%
63.3%
61.7%
70.8%
51.7%
-
46.7%
66.7%
68.8%
-79
2023 Houston R16
L vsBrouwer
50.9%
66.7%
60.0%
58.2%
59.4%
78.1%
69.2%
72.5%
53.8%
52.6%
0.0%
45.5%
59.1%
50.0%
103
2023 Dallas F
L vsWu
64.4%
71.2%
41.5%
67.0%
71.9%
77.5%
32.0%
75.4%
40.0%
48.1%
50.0%
42.9%
53.6%
56.3%
-27
2023 Australian Open R128
L vsMannarino
43.0%
56.3%
43.7%
49.1%
53.3%
65.6%
45.2%
62.1%
61.0%
61.2%
0.0%
19.4%
36.1%
41.4%
1
2022 Paris R32
L vsRublev
37.5%
50.0%
20.0%
42.0%
47.7%
59.1%
21.7%
53.4%
60.0%
42.1%
33.3%
0.0%
16.7%
16.7%
-118
2022 US Open R128
W vsDelbonis
61.1%
72.2%
42.9%
65.6%
72.5%
80.4%
50.0%
78.8%
47.4%
46.9%
-
33.3%
52.4%
35.7%
-79
2022 Cincinnati QF
L vsTsitsipas
41.8%
61.2%
50.9%
50.9%
55.1%
73.9%
54.8%
68.0%
50.0%
65.7%
100.0%
11.1%
33.3%
45.8%
-4
2022 Cincinnati R32
W vsHurkacz
46.3%
60.2%
50.0%
52.7%
58.5%
72.3%
59

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,48.1%,63.0%,55.7%,55.6%,62.2%,77.5%,64.3%,73.1%,57.1%,40.3%,0.0%,19.6%,34.8%,45.9%,69,2023,US Open,R64,L,Mmoh
1,44.6%,65.1%,65.2%,56.3%,43.4%,64.2%,63.3%,61.7%,70.8%,51.7%,-,46.7%,66.7%,68.8%,-79,2023,US Open,R128,W,Diaz
2,50.9%,66.7%,60.0%,58.2%,59.4%,78.1%,69.2%,72.5%,53.8%,52.6%,0.0%,45.5%,59.1%,50.0%,103,2023,Houston,R16,L,Brouwer
3,64.4%,71.2%,41.5%,67.0%,71.9%,77.5%,32.0%,75.4%,40.0%,48.1%,50.0%,42.9%,53.6%,56.3%,-27,2023,Dallas,F,L,Wu
4,43.0%,56.3%,43.7%,49.1%,53.3%,65.6%,45.2%,62.1%,61.0%,61.2%,0.0%,19.4%,36.1%,41.4%,1,2023,Australian Open,R128,L,Mannarino
5,37.5%,50.0%,20.0%,42.0%,47.7%,59.1%,21.7%,53.4%,60.0%,42.1%,33.3%,0.0%,16.7%,16.7%,-118,2022,Paris,R32,L,Rublev
6,61.1%,72.2%,42.9%,65.6%,72.5%,80.4%,50.0%,78.8%,47.4%,46.9%,-,33.3%,52.4%,35.7%,-79,2022,US Open,R128,W,Delbonis
7,41.8%,61.2%,50.9%,50.9%,55.1%,73.9%,54.8%,68.0%,50.0%,65.7%,100.0%,11.1%,33.3%,45.8%,-4,2022,Cincinnati,QF,L,Tsitsipas
8,46.3%,60.2%,50.0%,52.7%,58.5%,72.3%,59.3%,69.1%,55.6%,52.6%,0.0%,30.8%,46.2%,40.7%,62,2022,Cincinnati,R32,W,Hurkacz
9,37.2%,46.2%,47.7%,43.5%,50.0%,60.0%,52.0%,59.8%,32.0%,56.0%,50.0%,17.4%,26.1%,42.1%,39,2022,Newport,SF,L,Cressy


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/isner_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=104545/John-Isner&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2023 US Open R64
L vsMmoh
115.6
119.8
8.6
122.4
119.2
134
68
105.1
5.1
104.8
103.8
114
93
2023 US Open R128
W vsDiaz Acosta
117.1
120.3
5.4
123.0
118.9
132
108
111.3
5.7
114.6
109.0
122
93
2022 US Open R128
W vsDelbonis
118.8
123.0
3.8


129
117
104.0
3.0


107
101
2022 Wimbledon R32
L vsSinner
118.3
122.1
7.3
126.5
119.5
136
95
109.9
6.0
115.0
105.6
123
97
2022 Wimbledon R64
W vsMurray
124.8
127.9
6.1
130.0
125.0
138
114
115.7
4.3
118.3
115.2
122
103
2022 Wimbledon R128
W vsCouacaud
124.4
125.7
6.4
127.5
123.3
136
106
120.1
4.3
121.8
120.9
129
111
2021 US Open R128
L vsNakashima
119.9
122.7
7.3
127.5
122.4
142
107
113.6
7.2
115.3
110.5
131
95
2021 Wimbledon R128
L vsNishioka
120.0
123.5
6.6
125.6
120.5
134
109
114.0
6.1
118.3
111.8
124
94
2020 US Open R128
L vsJohnson
118.6
121.5
9.7
125.5
113.8
137
77
106.7
8.1
118.0
102.8
121
93
2019 US Open R32
L vsCilic
119.7
122.9
7.4
125.2
121.1
137
106
110.8
8.6
104.0
110.0
131
89
2019 US Open R64
W vsStruff
116.8
120.4
14.8
125.8
120.0
137
83


In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,115.6,119.8,8.6,122.4,119.2,134,68,105.1,5.1,104.8,103.8,114,93,2023,US Open,R64,L,Mmoh
1,117.1,120.3,5.4,123.0,118.9,132,108,111.3,5.7,114.6,109.0,122,93,2023,US Open,R128,W,Diaz
2,118.8,123.0,3.8,,,129,117,104.0,3.0,,,107,101,2022,US Open,R128,W,Delbonis
3,118.3,122.1,7.3,126.5,119.5,136,95,109.9,6.0,115.0,105.6,123,97,2022,Wimbledon,R32,L,Sinner
4,124.8,127.9,6.1,130.0,125.0,138,114,115.7,4.3,118.3,115.2,122,103,2022,Wimbledon,R64,W,Murray
5,124.4,125.7,6.4,127.5,123.3,136,106,120.1,4.3,121.8,120.9,129,111,2022,Wimbledon,R128,W,Couacaud
6,119.9,122.7,7.3,127.5,122.4,142,107,113.6,7.2,115.3,110.5,131,95,2021,US Open,R128,L,Nakashima
7,120.0,123.5,6.6,125.6,120.5,134,109,114.0,6.1,118.3,111.8,124,94,2021,Wimbledon,R128,L,Nishioka
8,118.6,121.5,9.7,125.5,113.8,137,77,106.7,8.1,118.0,102.8,121,93,2020,US Open,R128,L,Johnson
9,119.7,122.9,7.4,125.2,121.1,137,106,110.8,8.6,104.0,110.0,131,89,2019,US Open,R32,L,Cilic


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2023,US Open,R64,L,Mmoh,115.6,119.8,8.6,122.4,119.2,134,68,105.1,5.1,104.8,103.8,114,93
1,2023,US Open,R128,W,Diaz,117.1,120.3,5.4,123.0,118.9,132,108,111.3,5.7,114.6,109.0,122,93
2,2022,US Open,R128,W,Delbonis,118.8,123.0,3.8,NaN,NaN,129,117,104.0,3.0,NaN,NaN,107,101
3,2022,Wimbledon,R32,L,Sinner,118.3,122.1,7.3,126.5,119.5,136,95,109.9,6.0,115.0,105.6,123,97
4,2022,Wimbledon,R64,W,Murray,124.8,127.9,6.1,130.0,125.0,138,114,115.7,4.3,118.3,115.2,122,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2011,Wimbledon,R128,W,Mahut,106.4,110.3,11.7,NaN,NaN,134,81,97.9,10.1,NaN,NaN,125,84
89,2011,Roland Garros,R128,L,Nadal,121.5,128.3,7.8,NaN,NaN,144,111,106.0,6.3,NaN,NaN,123,94
90,2011,Australian Open,R32,L,Cilic,118.3,123.8,7.9,NaN,NaN,141,101,104.4,7.7,NaN,NaN,125,89
91,2011,Australian Open,R64,W,Stepanek,122.4,125.5,7.6,NaN,NaN,139,100,109.6,7.5,NaN,NaN,123,92


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/isner_serve_speed.csv', index=False) #change the name of the file to save it as you want